In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_3/models/B5R2b5_AN_1FC_fold3_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold3.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4598, 25)
(4598, 25)
Normal:  (3003, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4598 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.17771949, -0.12076534,  0.12924182, ..., -0.04825108,
        -0.00705942, -0.14018615],
       [-0.05433313, -0.15362805,  0.28513959, ..., -0.04078432,
         0.01708839, -0.07454488],
       [-0.12773648, -0.13352278,  0.28501913, ..., -0.10758203,
         0.03169828, -0.06760362],
       ...,
       [ 0.86769992,  0.59384263, -0.04185282, ...,  0.50979209,
         0.01300936,  0.48425332],
       [ 0.54877049,  0.51605141, -0.09407436, ...,  0.56234539,
        -0.18688789,  0.97953099],
       [ 0.1214546 ,  0.61834753,  0.06334729, ...,  0.68577349,
        -0.19045003,  0.22471833]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.177719,-0.120765,0.129242,0.184727,-0.206808,-0.177978,0.170139,-0.179523,-0.194136,0.019570,...,-0.141705,-0.069519,-0.210276,-0.119517,-0.145761,0.035651,-0.189395,-0.048251,-0.007059,-0.140186
1,-0.054333,-0.153628,0.285140,0.147967,-0.213575,-0.185668,0.371937,-0.199365,-0.137477,0.084903,...,-0.137058,0.055156,-0.162696,-0.032310,-0.160517,0.024291,-0.159126,-0.040784,0.017088,-0.074545
2,-0.127736,-0.133523,0.285019,0.336072,-0.197123,-0.146144,0.533975,-0.212546,-0.184665,0.193159,...,-0.125473,0.168939,-0.228763,-0.061977,-0.197558,0.060741,-0.110988,-0.107582,0.031698,-0.067604
3,-0.165007,-0.145555,0.238389,0.144050,-0.176887,-0.105909,0.212358,-0.121396,-0.191720,0.012289,...,-0.117185,0.038200,-0.089139,-0.049494,-0.187168,0.087113,-0.150927,-0.091628,-0.031848,-0.152177
4,-0.111493,-0.143136,0.302416,0.167051,-0.163495,-0.099826,0.343424,-0.177297,-0.147886,0.056150,...,-0.127792,-0.048486,-0.129867,-0.102956,-0.082834,0.081174,-0.119735,-0.103707,-0.000305,-0.144316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4593,1.061843,1.309959,-0.070865,-0.135632,0.843428,0.645690,-0.192677,0.254312,0.778489,-0.203189,...,0.364337,-0.158869,0.837498,1.266026,0.674074,-0.186404,0.749910,0.645284,-0.182605,0.589363
4594,0.587666,0.270031,-0.131149,-0.026325,0.449543,0.599538,-0.127480,0.429058,0.341174,-0.173343,...,0.474394,-0.129520,0.481323,0.748103,0.622545,-0.185966,0.633890,0.136641,-0.135174,0.473392
4595,0.867700,0.593843,-0.041853,0.605483,0.498619,0.888413,-0.124793,0.391564,0.358154,-0.209960,...,0.832881,-0.118444,0.188782,0.700182,0.556143,-0.196834,0.740996,0.509792,0.013009,0.484253
4596,0.548770,0.516051,-0.094074,-0.102425,0.704568,0.559603,-0.189042,0.332188,0.538850,-0.210648,...,0.439064,-0.201867,0.302577,0.660169,0.387179,-0.084772,0.720836,0.562345,-0.186888,0.979531


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.177719,-0.120765,0.129242,0.184727,-0.206808,-0.177978,0.170139,-0.179523,-0.194136,0.019570,...,-0.145761,0.035651,-0.189395,-0.048251,-0.007059,-0.140186,Abnormal,P31,FP-B,AB01
1,-0.054333,-0.153628,0.285140,0.147967,-0.213575,-0.185668,0.371937,-0.199365,-0.137477,0.084903,...,-0.160517,0.024291,-0.159126,-0.040784,0.017088,-0.074545,Abnormal,P1,FP-A,AB01
2,-0.127736,-0.133523,0.285019,0.336072,-0.197123,-0.146144,0.533975,-0.212546,-0.184665,0.193159,...,-0.197558,0.060741,-0.110988,-0.107582,0.031698,-0.067604,Abnormal,P2,FP-A,AB01
3,-0.165007,-0.145555,0.238389,0.144050,-0.176887,-0.105909,0.212358,-0.121396,-0.191720,0.012289,...,-0.187168,0.087113,-0.150927,-0.091628,-0.031848,-0.152177,Abnormal,P42,FP-B,AB01
4,-0.111493,-0.143136,0.302416,0.167051,-0.163495,-0.099826,0.343424,-0.177297,-0.147886,0.056150,...,-0.082834,0.081174,-0.119735,-0.103707,-0.000305,-0.144316,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4593,1.061843,1.309959,-0.070865,-0.135632,0.843428,0.645690,-0.192677,0.254312,0.778489,-0.203189,...,0.674074,-0.186404,0.749910,0.645284,-0.182605,0.589363,Normal,P32,FP-B,Normal
4594,0.587666,0.270031,-0.131149,-0.026325,0.449543,0.599538,-0.127480,0.429058,0.341174,-0.173343,...,0.622545,-0.185966,0.633890,0.136641,-0.135174,0.473392,Normal,P42,FP-B,Normal
4595,0.867700,0.593843,-0.041853,0.605483,0.498619,0.888413,-0.124793,0.391564,0.358154,-0.209960,...,0.556143,-0.196834,0.740996,0.509792,0.013009,0.484253,Normal,P52,FP-C,Normal
4596,0.548770,0.516051,-0.094074,-0.102425,0.704568,0.559603,-0.189042,0.332188,0.538850,-0.210648,...,0.387179,-0.084772,0.720836,0.562345,-0.186888,0.979531,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold3_1.csv') #เปลี่ยนชื่อไฟล์